In [9]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST

from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics, bwt_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive,EWC, LwF
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, Resize
import numpy as np
import random
import wandb
import timm
import torch
import avalanche

In [2]:
seed = 2023
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)


In [3]:

train_transform = Compose([
    Resize(224),
    ToTensor(),
])

test_transform = Compose([
    Resize(224),
    ToTensor(),
])


benchmark = avalanche.benchmarks.SplitCIFAR100(n_experiences=10, return_task_id=False, seed=seed, 
                       train_transform=train_transform,eval_transform = test_transform
                      )

Files already downloaded and verified
Files already downloaded and verified


In [4]:
benchmark.original_classes_in_exp

[{5, 9, 23, 37, 39, 48, 58, 62, 86, 96},
 {7, 18, 19, 28, 33, 41, 67, 69, 78, 99},
 {17, 29, 36, 44, 47, 66, 74, 80, 83, 88},
 {6, 20, 21, 27, 43, 51, 81, 82, 85, 90},
 {1, 4, 14, 63, 65, 73, 84, 91, 93, 95},
 {2, 10, 12, 32, 53, 54, 59, 75, 89, 92},
 {31, 34, 35, 40, 42, 45, 55, 64, 97, 98},
 {0, 8, 24, 38, 49, 52, 68, 71, 79, 87},
 {15, 16, 22, 25, 30, 50, 56, 57, 61, 94},
 {3, 11, 13, 26, 46, 60, 70, 72, 76, 77}]

In [5]:
benchmark.get_reproducibility_data()

{'class_ids_from_zero_from_first_exp': False,
 'class_ids_from_zero_in_each_exp': False,
 'class_mapping': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99],
 'classes_order': [39,
  48,
  62,
  58,
  37,
  9,
  5,
  86,
  96,
  23,
  41,
  69,
  18,
  7,
  67,
  78,
  99,
  28,
  19,
  33,
  17,
  36,
  29,
  80,
  88,
  44,
  66,
  47,
  83,
  74,
  51,
  81,
  90,
  20,
  82,
  27,
  85,
  6,
  43,
  21,
  95,
  14,
  84,
  1,
  73,
  91,
  4,
  93,
  63

In [6]:
# model = timm.models.vit_tiny_patch16_224(pretrained=True, num_classes=benchmark.n_classes)
model =timm.models.resnet18(pretrained= True, num_classes = benchmark.n_classes)
model = torch.nn.DataParallel(model)

In [7]:
model

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(

In [8]:
# loggers 
loggers = []

# log to Tensorboard
loggers.append(TensorboardLogger())

# log to text file
loggers.append(TextLogger(open('log.txt', 'a')))

# print to stdout
loggers.append(InteractiveLogger())

# W&B logger - comment this if you don't have a W&B account
loggers.append(WandBLogger(project_name="avalanche", run_name="ewc-SplitCifar100-Resnet18"))

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
#     timing_metrics(epoch=True, epoch_running=True),
#     cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=True,
                             stream=True),
#     disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=loggers
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: grbagwe. Use `wandb login --relogin` to force relogin


In [10]:
# CREATE THE STRATEGY INSTANCE (EWC)
# cl_strategy = EWC(
#     model,
#     optimizer=SGD(model.parameters(), lr=0.0001),
#     criterion=CrossEntropyLoss(),
#     ewc_lambda=0.7,
#     train_mb_size=50, train_epochs=10,
#     eval_mb_size=50,device='cuda',
#     evaluator=eval_plugin)

cl_strategy = LwF(
    model,
    optimizer= torch.optim.Adam(model.parameters(), lr=0.001),
    criterion=CrossEntropyLoss(),
    alpha=1, 
    temperature=2, 
    train_mb_size=200, train_epochs=70,
    eval_mb_size=100,device='cuda',
    evaluator=eval_plugin)



In [11]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start training on experience ", experience.current_experience)

    cl_strategy.train(experience)
    print("End training on experience", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(cl_strategy.eval(benchmark.test_stream[:]))
        
        
# for experience in benchmark.train_stream:
#     # train returns a dictionary which contains all the metric values
#     res = cl_strategy.train(experience)
#     print('Training completed')

#     print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
#     results.append(cl_strategy.eval(benchmark.test_stream))
    


Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:29<00:00,  1.16s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 4.8847
	Loss_MB/train_phase/train_stream/Task000 = 4.8383
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0022
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0050
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:14<00:00,  1.71it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 4.7604
	Loss_MB/train_phase/train_stream/Task000 = 4.7072
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0044
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0050
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:17<00:00,  1.41it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 4.6403
	Loss_MB/train_phase/train_stream/Task000 = 4.6067
	Top1

100%|████████████████████████████████████████████████████████████████████| 25/25 [00:16<00:00,  1.48it/s]
Epoch 24 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.7152
	Loss_MB/train_phase/train_stream/Task000 = 2.6643
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3878
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4250
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:12<00:00,  1.93it/s]
Epoch 25 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6606
	Loss_MB/train_phase/train_stream/Task000 = 2.5700
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4100
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4550
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:15<00:00,  1.62it/s]
Epoch 26 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6050
	Loss_MB/train_phase/train_stream/Task000 = 2.5587
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4340
	Top1_Acc_MB/train_phase/train_stre

	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.48it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp002 = 6.3431
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.0000
-- Starting eval on experience 3 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.47it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp003 = 6.3865
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003 = 0.0000
-- Starting eval on experience 4 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.76it/s]
> Eval on experience 4 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_strea

	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0018
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:18<00:00,  1.35it/s]
Epoch 15 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.8887
	Loss_MB/train_phase/train_stream/Task000 = 3.8502
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0028
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:20<00:00,  1.24it/s]
Epoch 16 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.7774
	Loss_MB/train_phase/train_stream/Task000 = 3.6709
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0076
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0050
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:21<00:00,  1.18it/s]
Epoch 17 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.6695
	Loss_MB/train_phase/train_stre

	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.3600
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:15<00:00,  1.61it/s]
Epoch 39 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3037
	Loss_MB/train_phase/train_stream/Task000 = 2.2856
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4582
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4150
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:15<00:00,  1.60it/s]
Epoch 40 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2706
	Loss_MB/train_phase/train_stream/Task000 = 2.1834
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4748
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4950
100%|████████████████████████████████████████████████████████████████████| 25/25 [00:15<00:00,  1.64it/s]
Epoch 41 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2346
	Loss_MB/train_phase/train_stream/Task000 = 2.1652
	Top1_Acc_Epoch/train_phase/train_stre

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:09<00:00,  2.73it/s]
Epoch 36 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6801
	Loss_MB/train_phase/train_stream/Task000 = 2.6487
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4310
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4450
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:09<00:00,  2.77it/s]
Epoch 37 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6145
	Loss_MB/train_phase/train_stream/Task000 = 2.5655
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4636
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4950
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████

	ExperienceForgetting/eval_phase/test_stream/Task000/Exp001 = 0.6760
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 6.1491
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.16it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp002 = -0.6560
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp002 = 0.6560
	Loss_Exp/eval_phase/test_stream/Task000/Exp002 = 6.0155
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.0000
-- Starting eval on experience 3 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:09<00:00,  2.51it/s]
Epoch 25 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.6400
	Loss_MB/train_phase/train_stream/Task000 = 3.5592
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0566
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0650
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:09<00:00,  2.54it/s]
Epoch 26 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.5277
	Loss_MB/train_phase/train_stream/Task000 = 3.5235
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0812
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.1000
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:09<00:00,  2.54it/s]
Epoch 61 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.7046
	Loss_MB/train_phase/train_stream/Task000 = 1.7680
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6762
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.6200
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:10<00:00,  2.49it/s]
Epoch 62 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.6839
	Loss_MB/train_phase/train_stream/Task000 = 1.6386
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6740
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.6600
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████

-- Starting eval on experience 7 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.19it/s]
> Eval on experience 7 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp007 = -0.7470
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp007 = 0.7470
	Loss_Exp/eval_phase/test_stream/Task000/Exp007 = 4.0589
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp007 = 0.0090
-- Starting eval on experience 8 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.74it/s]
> Eval on experience 8 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp008 = -0.6850
	ExperienceForgetting/eval_

In [12]:
import wandb
wandb.finish()

ExperienceBWT/eval_phase/test_stream/Task000/Exp000,▁▁▁▁▁▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp001,▁▁▁▁▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp002,█▁▁▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp003,█▁▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp004,█▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp005,█▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp006,█▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp007,█▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp008,▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp000,▁▁▁▁▁▁▁▁▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp001,▁▁▁▁▁▁▁▁
